In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import EarlyStoppingCallback

In [2]:
import sys
import gc
sys.path.insert(1, '/kaggle/input/cs433tweets/')
from kaggle_secrets import UserSecretsClient
import helpers as hlp
import wandb

user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb")
wandb.login(key=secret_value_0)

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

[nltk_data] Downloading package words to /usr/share/nltk_data...
[nltk_data]   Package words is already up-to-date!
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


There are 2 GPU(s) available.
Device name: Tesla T4


In [3]:
 # Load data and set labels
data_complaint = pd.read_table("/kaggle/input/cs433tweets/train_neg.txt", header=None, names=['tweet'], dtype=str,on_bad_lines='skip')
data_complaint['label'] = 0
data_non_complaint = pd.read_table("/kaggle/input/cs433tweets/train_pos.txt", header=None, names=['tweet'], dtype=str,on_bad_lines='skip')
data_non_complaint['label'] = 2

# Concatenate complaining and non-complaining data
data = pd.concat((data_complaint, data_non_complaint)).reset_index(drop=True)

In [4]:
data = hlp.transformer_processing(data)

100%|██████████| 196970/196970 [00:00<00:00, 197941.80it/s]


In [5]:
# Define pretrained tokenizer and model
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

Downloading:   0%|          | 0.00/929 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
X = data.tweet.values.tolist()
y = data.label.values.tolist()

del data
gc.collect()

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=2022)

del X
del y
gc.collect()

X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=300)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=300)


del X_train
del X_val
gc.collect()

0

In [7]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [8]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [9]:
def get_max(pred):
    if pred[0] > pred[2]: 
        return 0
    return 2

In [10]:
def compute_metrics(p):
    pred, labels = p
    #pred = np.argmax(pred, axis=1)
    pred = np.apply_along_axis(get_max, 1, pred)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    #recall = recall_score(y_true=labels, y_pred=pred)
    #precision = precision_score(y_true=labels, y_pred=pred)
    #f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy}

In [11]:
args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=4,
    seed=2022, 
    save_total_limit = 2,
    save_strategy = 'no',
    load_best_model_at_end=False)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics)

In [12]:
# Train pre-trained model
trainer.train()
trainer.save_model()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 158859
  Num Epochs = 4
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 9932
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: jdidio. Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Accuracy
500,0.342100,0.308733,0.870213
1000,0.317200,0.302887,0.871233
1500,0.313600,0.293987,0.873612
2000,0.300400,0.302404,0.870383
2500,0.293900,0.325180,0.877408
3000,0.218600,0.310009,0.879617
3500,0.225000,0.315728,0.880127
4000,0.225800,0.294903,0.881203
4500,0.224300,0.300210,0.883639
5000,0.218600,0.341623,0.882450


***** Running Evaluation *****
  Num examples = 17652
  Batch size = 64
/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 17652
  Batch size = 64
/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 17652
  Batch size = 64
/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to g

In [13]:
wandb.finish()

eval/accuracy,▁▂▃▁▅▆▆▇█▇▆▆▇▆▇▆▅▆▆
eval/loss,▂▁▁▁▂▂▂▁▁▃▄▄▃▃█▆▆▆▇
eval/runtime,█▁▂▃▂▇▃▁▂▂▂▄▃▃█▂▂▂▂
eval/samples_per_second,▁█▇▆▇▂▆█▇▇▇▅▆▆▁▇▇▇▇
eval/steps_per_second,▁█▇▆▇▂▆█▇▇▇▅▆▆▁▇▇▇▇
train/epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,██▇▇▆▆▆▅▅▄▄▄▃▃▃▂▂▁▁
train/loss,█▇▇▇▇▅▅▅▅▅▃▃▂▂▂▁▁▁▁
train/total_flos,▁
train/train_loss,▁
